In [1]:
import numpy as np
import plotly.express as px

def calc_p_impact(x,y,original_swap_x,fee):
    """
    @notice calculate slippage when selling x or y to the open market.
    Assuming that we trade against Uniswap is a conservative assumption
    """
    # TODO: Also need to incorporate original_swap_y in case we are selling collateral
    # TODO: Need to incorporate concentrated liquidity (e.g., no more liquidity beyond ]a, b[)
    # TODO: Need to incorproate the StableSwap invariant for crvUSD pool liquidity
    # x = 2e6 
    # y = 1e3
    k = x*y
    original_price = x/y
    # original_swap_x = 10e3
    # fee=0.00
    swap_x = original_swap_x*(1-fee)
    new_x = x + swap_x
    new_y = k/new_x
    swap_y = y-new_y
    trade_price = swap_x/swap_y
    new_price = new_x/new_y
    return(trade_price)

calc_p_impact(2e6,1e3,10e3,0.00) 
x = np.linspace(1,2e6, num=500)
y = [calc_p_impact(2e6,1e3,x_i,0.00) for x_i in x]
fig = px.scatter(x=x, y=y)
fig.show()

In [2]:
# Empirical slippage analysis
import csv
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

with open("weth_usdc_005_tier_trades.csv","r") as infile:
    data = pd.read_csv(infile)

with open("weth_usdc_005_tier_trades2.csv","r") as infile:
    data2 = pd.read_csv(infile)

data3 = pd.concat([data,data2]).drop_duplicates()
data3["WETH"]=data3["amount1"].apply(lambda x: int(x) / 1e18)
data3["USDC"]=data3["amount0"].apply(lambda x: int(x)/1e6)
data3["price_implied"] = data3["price_implied"]/1e18
data3["price_actual"] = data3["price_actual"]/1e18
data3["price_impact"]=abs(data3["price_implied"]-data3["price_actual"])/data3["price_implied"]
data3["evt_block_time"] = pd.to_datetime(data3['evt_block_time'], utc=True)
data3['unix_timestamp'] = data3['evt_block_time'].astype(int) // 10**9

slope, intercept = np.polyfit(data3["WETH"], data3["price_impact"], 1)

#Linear regression
# Generate Y-values based on the regression model
y_fit = slope * data3["WETH"] + intercept
plt.scatter(x=data3["WETH"], y=data3["price_impact"], label='Data', color='blue')
plt.plot(data3["WETH"], y_fit, label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', color='red')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

## Quadratic Regression
a, b, c = np.polyfit(data3["WETH"], data3["price_impact"], 2)
x_fit = np.linspace(min(data3["WETH"]), max(data3["WETH"]), 500)
y_fit = a * x_fit**2 + b * x_fit + c
plt.scatter(x=data3["WETH"], y=data3["price_impact"], label='Data', color='blue')
plt.plot(x_fit, y_fit,label=f'Fit: y = {a:.2f}x^2 + {b:.2f}x + {c:.2f}', color='red')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

print(a,b,c)

FileNotFoundError: [Errno 2] No such file or directory: 'weth_usdc_005_tier_trades.csv'

In [ ]:
# new slippage function based on empirical analysis
def slippage(x,fee):
    """
    @notice calculate slippage when selling x or y to the open market.
    Assuming that we trade against Uniswap is a conservative assumption
    """
    a,b,c = (2.602424822058012e-10,2.3009044333523045e-07,0.0004613261845851614)
    return(a*x**2+b*x+c)

slippage_x = np.logspace(-6,6, num=50, endpoint=True, base=10.0, dtype=None, axis=0)
print(slippage_x)
slippage_y = [slippage(x,0.00) for x in slippage_x]
fig=px.line(x=slippage_x, y=slippage_y)
fig.show()